In [6]:
!pip install --quiet -U langchain_openai langchain_core langgraph


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///amar-shop.db")

print(db.dialect)
print(db.get_usable_table_names())

sqlite
['products', 'services']


In [3]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool

@tool
def sql_agent_tool(query: str):
    """
    Executes an SQL query using an AI-powered agent to fetch relevant information from the database.

    Args:
        state (State): A dictionary containing the user's query and related context.

    Returns:
        State: A dictionary containing the response from the database and the tool invocation details.
    """

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
    response = agent_executor.invoke({"input": query})
    
    return response

llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([sql_agent_tool])

In [9]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

ImportError: cannot import name 'ToolNode' from 'langgraph.prebuilt' (unknown location)

In [4]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([sql_agent_tool]))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", END)

ImportError: cannot import name 'ToolNode' from 'langgraph.prebuilt' (unknown location)

In [ ]:
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [10]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Hello, what is 2 multiplied by 2? .")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Hello, what is 2 multiplied by 2? .
================================== Ai Message ==================================
Tool Calls:
  multiply (call_cL8dlqvKWR4sfwPLousTbOzl)
 Call ID: call_cL8dlqvKWR4sfwPLousTbOzl
  Args:
    a: 2
    b: 2
================================= Tool Message =================================
Name: multiply

4
